# Recommendation System for E-Commerce Website
This notebook implements a product recommendation system using collaborative filtering.
We use the features 'Users Purchased', 'Rating', and 'Price' to generate product similarity.

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("products_dataset.csv")
df.head()

,Product ID,Category,Rating,Users Purchased,Price
0,P0001,Bags,4.8,7976,5192.79
1,P0002,Men Wallet,1.9,9143,1362.10
2,P0003,Shoes,3.1,7059,3426.69
3,P0004,Earbuds,2.4,1966,1020.58
4,P0005,Sunglasses,1.3,5104,373.92


In [2]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the numerical features
scaler = MinMaxScaler()
df[['Rating', 'Users Purchased', 'Price']] = scaler.fit_transform(df[['Rating', 'Users Purchased', 'Price']])

# Compute a combined score
df['score'] = df['Rating'] * 0.4 + df['Users Purchased'] * 0.4 + (1 - df['Price']) * 0.2  # Assume lower price is better
df_sorted = df.sort_values(by='score', ascending=False)
df_sorted.head()

,Product ID,Category,Rating,Users Purchased,Price,score
673,P0674,Sunglasses,0.975,0.977791,0.006159,0.979885
912,P0913,Charger,0.950,0.997999,0.013290,0.976542
1141,P1142,Sunglasses,0.975,0.957683,0.003603,0.972353
993,P0994,Bags,0.950,0.997199,0.050671,0.968745
1476,P1477,Watches,0.975,0.953481,0.022387,0.966915


In [3]:
# Top N products recommendation
def recommend_top_products(n=5):
    return df_sorted.head(n)[['Product ID', 'Category', 'score']]

recommend_top_products()

,Product ID,Category,score
673,P0674,Sunglasses,0.979885
912,P0913,Charger,0.976542
1141,P1142,Sunglasses,0.972353
993,P0994,Bags,0.968745
1476,P1477,Watches,0.966915


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# Create a pivot table with products and features
features = df[['Rating', 'Users Purchased', 'Price']]
similarity_matrix = cosine_similarity(features)

# Function to get similar products
def recommend_similar_products(product_id, top_n=3):
    idx = df[df['Product ID'] == product_id].index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    return df.iloc[[i[0] for i in sim_scores]][['Product ID', 'Category', 'score']]

# Example
recommend_similar_products("P0001")

,Product ID,Category,score
1045,P1046,Watches,0.526778
292,P0293,Earbuds,0.529257
217,P0218,Shoes,0.415710


In [6]:
# Simple complementary category mapping
complementary_map = {
    "Watches": ["Sunglasses", "Men Wallet"],
    "Shoes": ["Bags"],
    "Smartphones": ["Earbuds"],
}

def recommend_complementary_by_category(category):
    if category not in complementary_map:
        return "No complementary recommendations available."
    related_categories = complementary_map[category]
    return df[df['Category'].isin(related_categories)].sort_values(by='score', ascending=False).head(5)

# Example
recommend_complementary_by_category("Smartphones")

,Product ID,Category,Rating,Users Purchased,Price,score
1466,P1467,Earbuds,0.975,0.902761,0.026390,0.945826
1998,P1999,Earbuds,0.875,0.975890,0.021704,0.936015
905,P0906,Earbuds,0.825,0.985494,0.009439,0.922310
1425,P1426,Earbuds,0.925,0.885154,0.032024,0.917657
1875,P1876,Earbuds,0.800,1.000000,0.025619,0.914876
